## Step 0: Configuring the environment

In this step, we are installing the libraries allowed for our project, which involve the use of LangChain, integration with Huggingface models, OpenAI, in addition to the storage of embeddings using ChromaDB.


In [1]:
%pip install -qU langchain-huggingface
!pip install promptquality
!pip install langchain
!pip install langchain_community
!pip install langchain-openai
!pip install chromadb
%pip install --upgrade --quiet  GitPython




[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 282.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 881.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 821.6 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 5.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: zipp
    Found existing installation: zipp 3.19.2
    Uninstalling zipp-3.19.2:
      Successfully uninstalled zipp-3.19.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 6.11.0
    Uninstalling importlib-metadata-6.11.0:
      Successfully uninst

## Local Environment
If you want to run models locally using Llama-cpp, just execute the following cell:

Learn more at: https://python.langchain.com/docs/integrations/llms/llamacpp/

In [25]:
# import os
# import pip

# os.environ["CUDACXX"] = "/usr/local/cuda-12/bin/nvcc"
# os.environ["CMAKE_ARGS"] = "-DLLAMA_CUBLAS=on"
# os.environ["FORCE_CMAKE"] = "1"
        
# pip.main(["install", "llama-cpp-python==0.2.55", "numpy==1.23"])

## Step 1: Cloning the Repository and Extracting Code from Jupyter Notebooks
In this step, we clone a GitHub repository, search for Jupyter Notebook files (*.ipynb*), and extract both code and context from these notebooks. The code performs the following operations:

-  **Cloning a GitHub Repository**:
We begin by cloning the desired repository from GitHub using the function clone_repo.
- Function clone_repo(*repo_url*, *clone_dir="./temp_repo"*):
  - **Objective**: This function clones a GitHub repository into a specified directory.
  - **Process**:
      - The function first checks if the target directory exists. If not, it creates the directory.
      - It then uses the git.Repo.clone_from method from the git Python module to clone the repository.
      - A confirmation message is printed to show where the repository has been cloned.
  - **Input**:
      - **repo_url**: The URL of the GitHub repository to be cloned.
      - **clone_dir**: The directory where the repository will be stored (default is ./temp_repo).

- **Locating All Notebooks in the Directory**:
Once the repository is cloned, we proceed to find all Jupyter Notebook files (.ipynb) within the cloned directory.
    - **Function** find_all_notebooks(directory):
    - **Objective**: This function recursively searches through the directory and identifies all files with the .ipynb extension.
    - **Process**: It uses os.walk() to traverse through the specified directory, listing all files and subdirectories.
For each file ending with .ipynb, the function adds the full file path to a list of notebooks.

- **Extracting Code and Context From Notebooks**:
  After locating the notebooks, the next step is to extract both the code and any markdown context from each notebook.
  - **Function** *extract_code_and_context(notebook_path)*
  - **Objective**: This function reads a notebook and extracts the code cells and any corresponding markdown context.

- **Process**:
  - The notebook is opened using the nbformat.read function.
  - The function iterates through each cell of the notebook:
  - If the cell is of type markdown, it extracts the content of the markdown cell as context.
  - If the cell is of type code, it creates a dictionary with the following fields:
    - **ID**: A unique identifier for the code snippet, generated using uuid.uuid4().
    - **Embedding**: Initially set to None (embeddings will be generated later).
    - **Code**: The code content of the cell.
    - **Filename**: The name of the notebook file.
    - **Context**: The markdown context associated with the code (if any).
The extracted code and context are appended to a list.



In [6]:
import os
import git
import nbformat
import uuid

# Function to clone GitHub repository
def clone_repo(repo_url, clone_dir="./temp_repo"):
    if not os.path.exists(clone_dir):
        os.makedirs(clone_dir)
    git.Repo.clone_from(repo_url, clone_dir)
    print(f"Repository cloned in: {clone_dir}")

In [7]:
# Function to find all .ipynb notebooks in a directory
def find_all_notebooks(directory):
    notebooks = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".ipynb"):
                notebooks.append(os.path.join(root, file))
    return notebooks


In [8]:
# Function to extract code and context from notebooks
def extract_code_and_context(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    extracted_data = []
    for cell in notebook['cells']:
        if cell['cell_type'] == 'markdown':
            context = ''.join(cell['source'])
        elif cell['cell_type'] == 'code':
            cell_data = {
                "id": str(uuid.uuid4()),  
                "embedding": None,        
                "code": ''.join(cell['source']),
                "filename": os.path.basename(notebook_path),
                "context": context if 'context' in locals() else ''
            }
            extracted_data.append(cell_data)

    return extracted_data


In [3]:
# Cloning the repository and performing the extraction
repo_url = "https://github.com/sergiopaniego/RAG_local_tutorial.git"  #your repository
clone_repo(repo_url)

Repository cloned in: ./temp_repo


In [9]:
# Locate and process notebooks
clone_dir = "./temp_repo"
notebooks = find_all_notebooks(clone_dir)

all_extracted_data = []

for notebook in notebooks:
    print(f"Extracting data from: {notebook}")
    extracted_data = extract_code_and_context(notebook)  
    all_extracted_data.extend(extracted_data)

print("Extraction completed.")



Extracting data from: ./temp_repo/example_rag.ipynb
Extracting data from: ./temp_repo/github_repo_rag.ipynb
Extracting data from: ./temp_repo/llm_chat_memory.ipynb
Extracting data from: ./temp_repo/whisper_rag.ipynb
Extracting data from: ./temp_repo/youtube_rag.ipynb
Extraction completed.


In [10]:
all_extracted_data

[{'id': '06a1bf36-1943-4fa4-bedc-6b0c9b6a7775',
  'embedding': None,
  'code': '!pip3 install langchain\n!pip3 install langchain_pinecone\n!pip3 install langchain[docarray]\n!pip3 install docarray\n!pip3 install pypdf',
  'filename': 'example_rag.ipynb',
  'context': '# Install the requirements\n\nIf an error is raised related to docarray, refer to this solution: https://stackoverflow.com/questions/76880224/error-using-using-docarrayinmemorysearch-in-langchain-could-not-import-docarray'},
 {'id': 'e27724b3-3607-427c-833b-9745b630d7d1',
  'embedding': None,
  'code': '#MODEL = "gpt-3.5-turbo"\n#MODEL = "mixtral:8x7b"\n#MODEL = "gemma:7b"\n#MODEL = "llama2"\nMODEL = "llama3" # https://ollama.com/library/llama3',
  'filename': 'example_rag.ipynb',
  'context': '# Select the LLM model to use\n\nThe model must be downloaded locally to be used, so if you want to run llama3, you should run:\n\n```\n\nollama pull llama3\n\n```\n\nCheck the list of models available for Ollama here: https://olla

## Step 2: Generate metadata with llm  🔢

In this step, we use a language model (LLM) to generate descriptions and explanatory metadata for each extracted code snippet. The code performs the following operations:

-  We define a prompt template that contains placeholders for the code snippet, the file name, and an optional context. The goal is for the model to provide a clear and concise explanation of what the code does, based on these three pieces of information.

-  A PromptTemplate object is created from this template, allowing it to be used in conjunction with the language model.

-  We use the OpenAI LLM, authenticated with an API key, to process the information and generate responses.

- The function update_context_with_llm iterates through the data structure containing the extracted code, runs the language model for each item, and replaces the original context field with the explanation generated by the AI.

- Finally, the data structure is updated with the new explanations, which are stored in the context field.

-  The ultimate goal is to enrich the original data structure by providing clear explanations for each code snippet, making it easier to understand and use the information later

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

In [50]:
os.environ["OPENAI_API_KEY"] = "" #your api key

In [13]:
template = """
You will receive three pieces of information: a code snippet, a file name, and an optional context. Based on this information, explain in a clear, summarized and concise way what the code snippet is doing.

Code:
{code}

File name:
{filename}

Context:
{context}

Describe what the code above does.
"""

prompt = PromptTemplate.from_template(template)



In [14]:
llm = OpenAI()

llm_chain = prompt | llm


### Generate metadata with llm local

If you happen to be using a local model with LlamaCPP to generate metadata

In [29]:
# from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
# from langchain_community.llms import LlamaCpp

# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# llm_local = LlamaCpp(
            # model_path="/home/jovyan/datafabric/Llama7b/ggml-model-f16-Q5_K_M.gguf",
            # n_gpu_layers=64,
            # n_batch=512,
            # n_ctx=4096,
            # max_tokens=1024,
            # f16_kv=True,  
            # callback_manager=callback_manager,
            # verbose=False,
            # stop=[],
            # streaming=False,
            # temperature=0.4,
        # )

# llm_chain = prompt | llm_local


In [15]:
def update_context_with_llm(data_structure):
    updated_structure = []
    for item in data_structure:
        code = item['code']
        filename = item['filename']
        context = item['context']
    
        # Run LLM to generate code explanation
        response = llm_chain.invoke({
            "code": code, 
            "filename": filename, 
            "context": context
        })
        
        # Update the item with the LLM response
        item['context'] = response.strip()
        updated_structure.append(item)
    
    return updated_structure

In [16]:
updated_data = update_context_with_llm(all_extracted_data)

In [17]:
updated_data

[{'id': '06a1bf36-1943-4fa4-bedc-6b0c9b6a7775',
  'embedding': None,
  'code': '!pip3 install langchain\n!pip3 install langchain_pinecone\n!pip3 install langchain[docarray]\n!pip3 install docarray\n!pip3 install pypdf',
  'filename': 'example_rag.ipynb',
  'context': 'The code snippet installs several packages using pip3 in order to run a file named "example_rag.ipynb". One of the packages, "docarray", may cause an error, so the code also includes a solution to address this potential issue.'},
 {'id': 'e27724b3-3607-427c-833b-9745b630d7d1',
  'embedding': None,
  'code': '#MODEL = "gpt-3.5-turbo"\n#MODEL = "mixtral:8x7b"\n#MODEL = "gemma:7b"\n#MODEL = "llama2"\nMODEL = "llama3" # https://ollama.com/library/llama3',
  'filename': 'example_rag.ipynb',
  'context': 'The code snippet is setting a variable, MODEL, to a specific value. The file name is example_rag.ipynb. The context provides instructions on how to select and use the LLM model, and also includes a link to a website where a li

## Step 3: Generate Embeddings and Structure Data

In this step, we use an embeddings model to generate embedding vectors for the context extracted from each code snippet. The code performs the following operations:

**HuggingFace Embeddings**: We use the HuggingFace embeddings model "all-MiniLM-L6-v2" to generate vectors that semantically represent the context of the code snippets.

**Function** *update_embeddings*: This function iterates through the previously extracted data structure. For each item:

- Generates an embedding vector from the context field using the embed_query method of the embeddings model.
- Updates the item in the data structure, inserting the new embedding vector into the embedding field.
Conversion to DataFrame: After updating the data structure with the embeddings, we use the to_dataframe_row function to convert the list of code snippets and their respective metadata into a format suitable for a Pandas DataFrame.

Each item in the data structure is converted into a dictionary containing:

- **ID**: A unique identifier for the code snippet.
- **Embeddings**: The embedding vector generated for the context.
- **Code**: The extracted code.
- **Metadata**: Additional metadata, such as the filename and updated context.
  
The list of dictionaries is then converted into a DataFrame.

Creating the DataFrame: The to_dataframe_row function organizes this data, and Pandas is used to create a DataFrame, facilitating the manipulation and future use of the data with the results stored in a DataFrame for easy visualization and further processing.

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-09-17 13:21:14.542555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-17 13:21:16.520573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
def update_embeddings(data_structure):
    updated_structure = []
    for item in data_structure:
        context = item['context']

        # Generate the embedding for the context
        embedding_vector = embeddings.embed_query(context)

        # Update the item with the new embedding
        item['embedding'] = embedding_vector
        updated_structure.append(item)
    
    return updated_structure


In [20]:
updated_structure = update_embeddings(all_extracted_data)

In [21]:
import pandas as pd

def to_dataframe_row(embedded_snippets: list):
    """
    Helper function to convert a list of embedded snippets into a dataframe row
    in dictionary format.

    Args:
        embedded_snippets: List of dictionaries containing Snippets to be converted

    Returns:
        List of Dictionaries suitable for conversion to a DataFrame
    """
    outputs = []
    for snippet in embedded_snippets:
        output = {
            "ids": snippet['id'],
            "embeddings": snippet['embedding'],
            "code": snippet['code'],
            "metadatas": {
                "filenames": snippet['filename'],
                "context": snippet['context'],
            },
        }
        outputs.append(output)
    return outputs




In [23]:
rows = to_dataframe_row(updated_structure)
df = pd.DataFrame(rows)

In [24]:
df

,ids,embeddings,code,metadatas
0,06a1bf36-1943-4fa4-bedc-6b0c9b6a7775,"[-0.012226501479744911, 0.07351084053516388, -...",!pip3 install langchain\n!pip3 install langcha...,"{'filenames': 'example_rag.ipynb', 'context': ..."
1,e27724b3-3607-427c-833b-9745b630d7d1,"[-0.04303930699825287, 0.00578521890565753, -0...","#MODEL = ""gpt-3.5-turbo""\n#MODEL = ""mixtral:8x...","{'filenames': 'example_rag.ipynb', 'context': ..."
2,c335cbc6-421b-40e9-acf0-042ac2b87dbc,"[-0.1125277429819107, 0.00823664665222168, -0....",from langchain_community.llms import Ollama\nf...,"{'filenames': 'example_rag.ipynb', 'context': ..."
3,f7671a48-7252-49ca-9bc4-7dd3771a5521,"[-0.035975854843854904, 0.019312957301735878, ...","model.invoke(""Waht is 2+2?"")","{'filenames': 'example_rag.ipynb', 'context': ..."
4,822093d3-820f-4621-a7da-a0c8892feadd,"[-0.07964743673801422, 0.06151750311255455, -0...",from langchain_core.output_parsers import StrO...,"{'filenames': 'example_rag.ipynb', 'context': ..."
...,...,...,...,...
65,c843c07c-ffe7-4932-b4f7-d3cfeb3b23d1,"[-0.0798792690038681, 0.033993981778621674, -0...",from langchain_community.vectorstores import D...,"{'filenames': 'youtube_rag.ipynb', 'context': ..."
66,d5d99d34-5b43-4120-8639-17e57be92cff,"[-0.06140899658203125, 0.058361735194921494, -...",retriever = vectorstore.as_retriever(),"{'filenames': 'youtube_rag.ipynb', 'context': ..."
67,b7573242-22c6-4e31-a315-71903c475b7d,"[-0.05073198303580284, -0.021852439269423485, ...",from langchain_core.output_parsers import StrO...,"{'filenames': 'youtube_rag.ipynb', 'context': ..."
68,e7d43995-2417-4f73-b013-00568a9bdd6a,"[-0.0349389873445034, 0.0597563199698925, -0.0...",from langchain.prompts import PromptTemplate\n...,"{'filenames': 'youtube_rag.ipynb', 'context': ..."


## Step 4: Store and Query Documents in ChromaDB 🔗🏦

In this step, we use ChromaDB, a vector database system, to store code snippets and their respective metadata. We also implement a function to retrieve documents based on queries. The code performs the following operations:

####  Connection and Collection Creation
- **ChromaDB Client**: A ChromaDB client is initialized to interact with the database.
- **Collection Creation or Retrieval**: The collection named "my_collection" is created (or retrieved, if it already exists) within the ChromaDB database. Collections are used to store documents and their corresponding embeddings.
#### Inserting Documents
- **Data Extraction**: The following fields are extracted from the DataFrame and converted into lists:
   - **ids**: A list of unique identifiers for each document (code snippet).
   - **documents**: A list of code snippets.
   - **metadatas**: A list of metadata associated with each document, such as the filename and context.
   - **embeddings_list**: A list of embedding vectors previously generated for the context of each code snippet.
- **Inserting into ChromaDB**: The upsert method is used to insert or update the documents, ids, metadata, and embeddings in the created collection.
#### Querying Documents
- **Query**: After adding the documents to the collection, a query is performed. The code searches for documents related to the query text "!pip install", returning the 5 most relevant results.
#### *retriever* **Function*
- **Document Retrieval**: The retriever function is implemented to query the collection. It takes a query string, the collection, and the number of results to return (top_n) as parameters.
  - **Query in ChromaDB**: The function executes a query in the collection using the provided string.
  - **Creating Document Objects**: For each result returned, the function creates a Document instance containing the page content (code snippet) and its metadata.
  - **Returning Documents**: The function returns a list of Document objects that contain the page content and metadata for easy retrieval and future analysis.


In [25]:
import chromadb

chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(name="my_collection")

ids = df["ids"].tolist()
documents = df["code"].tolist()
metadatas = df["metadatas"].tolist()
embeddings_list = df["embeddings"].tolist()

collection.upsert(
    documents=documents,
    ids=ids,
    metadatas=metadatas,
    embeddings=embeddings_list  
)

print("Documents added successfully!!")


Documents added successfully!!


In [26]:
document_count = collection.count()
print(f"Total documents in the collection: {document_count}")

Total documents in the collection: 70


In [27]:
results = collection.query(
    query_texts=["!pip install"],
    n_results=5,  
)

/home/jovyan/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:24<00:00, 3.39MiB/s]


In [28]:
results

{'ids': [['8f9b02a3-df36-437e-8538-ae760fae7d8e',
   '3dd6c81f-1930-4efe-b4e9-b6313cf617ab',
   '06a1bf36-1943-4fa4-bedc-6b0c9b6a7775',
   '49e92e04-2ebe-4cf5-8696-ee1bacd58f89',
   '1b8f0093-cc71-4df3-ad12-5438a4cf7ea5']],
 'distances': [[1.1053783893585205,
   1.1907209157943726,
   1.1907271146774292,
   1.1952283382415771,
   1.2564711570739746]],
 'metadatas': [[{'context': 'The code snippet is installing five different packages using the pip3 command. These packages are langchain, langchain_pinecone, docarray, and pypdf. The code is being executed in the file named "youtube_rag.ipynb". The purpose of this code is to install the necessary dependencies for a project related to YouTube and the RAG (RetrievAndGenerate) model.',
    'filenames': 'youtube_rag.ipynb'},
   {'context': 'The code snippet installs the openai-whisper package using pip3 and its dependencies. The file name is whisper_rag.ipynb, suggesting that it is a Jupyter notebook file related to the whisper package. The o

In [46]:
from langchain.schema import Document
from typing import List


def retriever(query: str, collection, top_n: int = 10) -> List[Document]:
    results = collection.query(
        query_texts=[query],
        n_results=top_n
    )
    
    documents = [
        Document(
            page_content=str(results['documents'][i]),
            metadata=results['metadatas'][i] if isinstance(results['metadatas'][i], dict) else results['metadatas'][i][0]  # Corrigir o metadado se for uma lista
        )
        for i in range(len(results['documents']))
    ]
    
    return documents


## Step 5: Chain 🦜⛓️

In this step, we use a flow to automatically generate Python code based on a provided context and question. The code performs the following:

#### Function *format_docs(docs: List[Document]) -> str:*
- **Purpose**: This function formats a list of documents docs into a single string by concatenating the content of each document (doc.page_content) with two line breaks (\n\n) between them. This ensures that the context used in code generation is organized and readable.

#### Language Model and Processing Chain:
- **ChatOpenAI**: A language model from OpenAI is used to generate responses based on the provided prompt.
- The **chain**processes data using the following components:
  - **Context**: The context is formatted using the *format_docs* function, which calls the retriever function to fetch relevant context from the document base.
  - **Question**: The question is passed directly through the chain to process the prompt.
  - **Model**: The model generates the code based on the template and the provided data.
  - **Output Parser**: The output is processed with StrOutputParser to ensure the return is a clean string.

#### Function *clean_and_print_code(result: str)*:
- Purpose: This function takes the generated code string from the model and removes any formatting markers (e.g., ```python). After cleaning, the code is printed in a clean format, ready for execution.

#### Interaction with Galileo:
- The *promptquality* library is used to evaluate the quality of the generated prompts.
- **Galileo Callback**: A custom callback is configured using the Galileo API Key, where the following evaluation scopes are set:
   - **Context Adherence**: Evaluates whether the generated code aligns with the provided context.
   - **Correctness**: Checks the factual accuracy of the generated code.
   - **Prompt Perplexity**: Measures the complexity of the prompt, useful for evaluating its clarity.
 
#### Chain Execution:
- A set of inputs containing the query and the question is provided to run the chain. The system generates code based on questions like "How can I use audio in RAG?" and "create code audio with RAG" using the vector base.

#### Results Publishing:
- The Galileo callback finalizes and publishes the results, recording the evaluation of each run of the code generation chain.

#### Function *create_new_code_cell_from_output(output)*:
 - Purpose: This function dynamically creates a new code cell in the Jupyter Notebook from the generated output. It handles different output formats such as strings or dictionaries (if the output contains JSON) and inserts the resulting code into the next code cell in the notebook.


#### Processing the results: 
- After the chain execution, the function iterates over each generated result, attempts to parse it as JSON, and creates a new code cell in the notebook from the output. If the result is not JSON, it treats the output as a code string.

In [42]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([doc.page_content for doc in docs])

template = """You are a Python wizard tasked with generating code for a Jupyter Notebook (.ipynb) based on the given context.
Your answer should consist of just the Python code, without any additional text or explanation.

Context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI() 

chain = {
    "context": lambda inputs: format_docs(retriever(inputs['query'], collection)), 
    "question": RunnablePassthrough()
} | prompt | model | StrOutputParser()




### Local Model
Cell to run a local model using LlamaCPP

In [ ]:
# def format_docs(docs: List[Document]) -> str:
    # return "\n\n".join([doc.page_content for doc in docs])

# template = """You are a Python wizard tasked with generating code for a Jupyter Notebook (.ipynb) based on the given context.
# Your answer should consist of just the Python code, without any additional text or explanation.

# Context:
# {context}

# Question: {question}
# """

# prompt = ChatPromptTemplate.from_template(template)
# model = llm_local() 

# chain = {
    # "context": lambda inputs: format_docs(retriever(inputs['query'], collection)), 
    # "question": RunnablePassthrough()
# } | prompt | model | StrOutputParser()

In [43]:
def clean_and_print_code(result: str):
    clean_code = result.replace("```python", "").replace("```", "").strip()
    
    print(clean_code)

In [44]:
import promptquality as pq

os.environ['GALILEO_API_KEY'] = "htMRukWlQyvOEDMnAUYQUTQnEZL6_3ubALGkhn6ph70" #your api Key
galileo_url = "https://console.hp.galileocloud.io/"
pq.login(galileo_url)


👋 You have logged into 🔭 Galileo (https://console.hp.galileocloud.io/) as diogo.vieira@hp.com.


Config(console_url=Url('https://console.hp.galileocloud.io/'), username=None, password=None, api_key=SecretStr('**********'), token=SecretStr('**********'), current_user='diogo.vieira@hp.com', current_project_id=None, current_project_name=None, current_run_id=None, current_run_name=None, current_run_url=None, current_run_task_type=None, current_template_id=None, current_template_name=None, current_template_version_id=None, current_template_version=None, current_template=None, current_dataset_id=None, current_job_id=None, current_prompt_optimization_job_id=None, api_url=Url('https://api.hp.galileocloud.io/'))

### Information Parameter 💡

**Query**: A query is generally used to retrieve information, such as documents or code snippets, from a database or retrieval system, like a vector database or an embeddings database. In this case, the query is likely being used to search for code snippets related to the specific request, such as the creation of an LLM model and an embedding model.

**Question**: The question represents the specific task you are asking the language model to perform. This involves generating code based on the context retrieved by the query. The question is sent to the LLM to generate the appropriate response or code based on the provided information.

In [47]:
from IPython.display import display, Markdown
from IPython import get_ipython
from IPython.display import display, Code


prompt_handler = pq.GalileoPromptCallback(
    scorers=[
        pq.Scorers.context_adherence_plus,  # groundedness
        pq.Scorers.correctness,             # factuality
        pq.Scorers.prompt_perplexity        # perplexity 
    ]
)

# Example of inputs to run the chain
inputs = [
    {"query": "instantiate the LLM model and the Embedding model", "question": "create code llm model and the embedding model"},

]
#How to create a vector bank?
#create code a chromadb vector database

results = chain.batch(inputs, config=dict(callbacks=[prompt_handler]))

# Publish run results
prompt_handler.finish()


Processing chain run...:   0%|          | 0/5 [00:00<?, ?it/s]

Initial job complete, executing scorers asynchronously. Current status:
cost: Done ✅
toxicity: Done ✅
pii: Done ✅
protect_status: Done ✅
prompt_perplexity: Failed ❌, error was: Executing this metric requires credentials for OpenAI or Azure OpenAI service to be set.
latency: Done ✅
groundedness: Failed ❌, error was: Executing this metric requires credentials for OpenAI, Azure OpenAI or Vertex to be set.
factuality: Failed ❌, error was: Executing this metric requires credentials for OpenAI, Azure OpenAI or Vertex to be set.
🔭 View your prompt run on the Galileo console at: https://console.hp.galileocloud.io/prompt/chains/81833064-c547-4e3c-b0f1-e99da1f7250c/3094a823-e59a-42ce-a09d-9f61f2deef0e?taskType=12


In [48]:
import json
from IPython.core.getipython import get_ipython

def create_new_code_cell_from_output(output):
    """
    Creates a new code cell in Jupyter Notebook from an output,
    dealing with different output formats.

    Args:
        output: The output to be inserted into the new cell. It can be a string, a dictionary
                or another type of object.
    """

    shell = get_ipython()

    if isinstance(output, dict):
        code = output['cells'][0]['source']
        code = ''.join(code)
    else:
        code = str(output)

    clean_code = code.strip()

    shell.set_next_input(clean_code, replace=False)

for result in results:
    try:
        output = json.loads(result)
        create_new_code_cell_from_output(output)
    except json.JSONDecodeError:
        # If it's not JSON, just treat it as a string of code
        create_new_code_cell_from_output(result)


### LLM generated code here!!!

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model="llama3")
embeddings = OllamaEmbeddings(model="llama3")